In [8]:
%autosave 10
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize, integrate
import astropy.units as u
import astropy.constants as c
from astropy import cosmology
from astroquery.vizier import Vizier

Autosaving every 10 seconds


In [7]:
vizier = Vizier(row_limit=1000)
tables = vizier.get_catalogs('J/A+A/568/A22/tablef3')
table = tables[0]
table

Name,zcmb,zhel,mb,e_mb,x1,e_x1,c,e_c,logMst,e_logMst,tmax,e_tmax,cov_mb_s_,cov_mb_c_,cov_s_c_,set,RAJ2000,DEJ2000,bias,SimbadName
,,,mag,mag,,,,,[Msun],[Msun],d,d,,,,,deg,deg,mag,
bytes12,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float32,float32,float32,float32,uint8,float64,float64,float32,bytes17
03D1au,0.503,0.5043,23.002,0.088,1.273,0.150,-0.012,0.030,9.517,0.110,52909.745,0.214,0.00079,0.00044,-0.00003,1,36.043210,-4.037469,0.002,SNLS 03D1au
03D1aw,0.581,0.5820,23.574,0.090,0.974,0.274,-0.025,0.037,9.169,0.088,52902.898,0.353,0.00282,0.00041,0.00157,1,36.061634,-4.517158,0.001,SNLS 03D1aw
03D1ax,0.495,0.4960,22.960,0.088,-0.729,0.102,-0.100,0.030,11.580,0.112,52915.924,0.112,0.00054,0.00047,-0.00002,1,36.097287,-4.720774,0.002,SNLS 03D1ax
03D1bp,0.346,0.3470,22.398,0.087,-1.155,0.113,-0.041,0.027,10.821,0.124,52920.249,0.103,0.00111,0.00062,0.00029,1,36.657235,-4.838779,0.000,SNLS 03D1bp
03D1co,0.678,0.6790,24.078,0.098,0.619,0.404,-0.039,0.067,8.647,0.284,52954.458,0.455,0.01186,0.00078,0.00590,1,36.567748,-4.935050,-0.003,SNLS 03D1co
03D1dt,0.611,0.6120,23.285,0.093,-1.162,1.641,-0.095,0.050,9.715,0.092,52962.253,0.977,0.02967,0.00095,0.04436,1,36.629968,-4.052341,0.000,SNLS 03D1dt
03D1ew,0.866,0.8680,24.354,0.106,0.376,0.348,-0.063,0.068,8.530,0.805,52991.742,0.665,0.00318,-0.00160,0.00409,1,36.058795,-4.665852,-0.018,SNLS 03D1ew
03D1fc,0.331,0.3320,21.861,0.086,0.650,0.119,-0.018,0.024,10.391,0.036,53002.764,0.104,0.00101,0.00053,0.00054,1,36.431648,-4.144059,-0.001,SNLS 03D1fc


In [27]:
z = table['zcmb']
mb = table['mb']
x1 = table['x1']
color = table['c']

def mu(mb, x1, color, Mb, alpha, beta):
    return mb - Mb + alpha*x1 - beta*color

def distance(z, H0, o_l):
    """Luminosity distance
    
    Paramters
    ---------
    z: float
        Redshift
    H0: float
        Hubble constant, km/s/Mpc
    o_l: float
        Omega_Lambda
        
    Returns
    -------
    float, pc
    """
    if not isinstance(z, np.ndarray):
        z = np.array([z])
    dist = np.empty(shape=z.shape)
    for i, zi in enumerate(z):
        d = integrate.quad(
            lambda z: 1. / np.sqrt(o_l + (1-o_l)*(1+z)**3),
            0, zi
        )[0] * (1+zi) * c.c.to_value(u.km/u.s) / H0
        dist[i] = d
    return 1e6 * dist

def mu_th(z, H0, o_l):
    return 5 * np.log10(distance(z, H0, o_l)) - 5

def test_distance():
    planck = cosmology.Planck15
    z = 1
    d1 = distance(
        z,
        planck.H0.to_value(u.km/u.s/u.Mpc),
        planck.Ode0
    )
    d2 = planck.luminosity_distance(z).to_value(u.pc)
    eps = 1e-3
    assert abs(d1-d2) < d1*eps

test_distance()

def chi2(H0, mb, x1, color, z):
    mu1 = mu(mb, x1, color, -19, 0.14, 3.1)
    mu2 = mu_th(z, H0, 0.7)
    return np.sum(np.square(mu1-mu2))

optimize.minimize(
   chi2,
   [50.0],
   args=(mb, x1, color, z),
)

      fun: 21.658101136713267
 hess_inv: array([[ 0.75077332]])
      jac: array([ -3.81469727e-06])
  message: 'Optimization terminated successfully.'
     nfev: 27
      nit: 7
     njev: 9
   status: 0
  success: True
        x: array([ 72.41636836])